# Disjoint-domain network

### Ethan Blackwood
### October 23, 2020

**Goal**: Train and analyze the network in Rogers/McClelland 2008 with 4 disjoint domains (Figures R3-R5), which learns to extract the feature of being more or less similar to other items in the same domain, across the 4 domains which have no items, contexts or attributes in common.

In [1]:
import numpy as np
from datetime import datetime as dt

import disjoint_domain as dd
import ddnet

Common training procedure:

In [4]:
def train_n_dd_nets(n=36, run_type='', net_params=None, train_params=None):

    # get some defaults
    ctx_per_domain, n_domains, n_items, n_ctx, attrs_per_context = dd.get_net_dims()
    device, torchfp = dd.init_torch()
    
    net_defaults = {
        'ctx_per_domain': ctx_per_domain,
        'attrs_per_context': attrs_per_context,
        'n_domains': n_domains,
        'device': device,
        'torchfp': torchfp
    }
    if net_params is None:
        net_params = {}
    net_params = {**net_defaults, **net_params}
    if net_params['device'].type == 'cuda':
        print('Using CUDA')
    else:
        print('Using CPU')
    
    train_defaults = {
        'lr': 0.01,
        'scheduler': None,
        'num_epochs': 3001,
        'batch_size': 16,
        'report_freq': 50,
        'snap_freq': 50,
        'snap_freq_scale': 'lin',
        'holdout_testing': 'none',
        'test_thresh': 0.97,
        'test_max_epochs': 10000,
        'reports_per_test': 4,
        'do_combo_testing': True
    }
    if train_params is None:
        train_params = {}
    train_params = {**train_defaults, **train_params}
    
    snaps_all = []
    reports_all = []
    
    for i in range(n):
        print(f'Training Iteration {i+1}')
        print('---------------------')
        
        net = ddnet.DisjointDomainNet(**net_params)
        snaps, reports = net.do_training(**train_params)
        snaps_all.append(snaps)
        reports_all.append(reports)

        print('')

    snaps = dict()
    for snap_type in snaps_all[0].keys():
        snaps[snap_type] = np.stack([snaps_one[snap_type] for snaps_one in snaps_all])
        
    reports = dict()
    for report_type in reports_all[0].keys():
        reports[report_type] = np.stack([reports_one[report_type] for reports_one in reports_all])
    
    if run_type != '':
        run_type += '_'

    np.savez(f'data/{run_type}dd_res_{dt.now():%Y-%m-%d_%H-%M-%S}',
             snapshots=snaps, reports=reports, net_params=net_params, train_params=train_params)

Base network: test on one combination of item/context for each domain.

In [5]:
train_n_dd_nets()

Using CUDA
Training Iteration 1
---------------------
Holding out: A4*/A1, B2*/B4, C6@/C3, D5@/D2
Epoch    0 end: loss = 208.480, acc = 0.586, test acc = 0.966
Epoch   50 end: loss = 110.484, acc = 0.965, test acc = 0.969
Epoch  100 end: loss = 109.579, acc = 0.968, test acc = 0.969
Epoch  150 end: loss = 109.093, acc = 0.969, test acc = 0.969
Epoch  200 end: loss = 108.801, acc = 0.969, test acc = 0.969
Epoch  250 end: loss = 108.914, acc = 0.969, test acc = 0.969
Epoch  300 end: loss = 108.707, acc = 0.969, test acc = 0.969
Epoch  350 end: loss = 108.652, acc = 0.969, test acc = 0.969
Epoch  400 end: loss = 108.456, acc = 0.969, test acc = 0.969
Epoch  450 end: loss =  94.005, acc = 0.929, test acc = 0.857
Epoch  500 end: loss =  82.498, acc = 0.892, test acc = 0.858
Epoch  550 end: loss =  74.956, acc = 0.871, test acc = 0.837
Epoch  600 end: loss =  67.817, acc = 0.876, test acc = 0.823
Epoch  650 end: loss =  62.717, acc = 0.877, test acc = 0.812
Epoch  700 end: loss =  58.303, ac

Network with merged representation layer (items & contexts all go to all rep units)

In [6]:
train_n_dd_nets(run_type='merged', net_params={'merged': True})

Using CUDA
Training Iteration 1
---------------------
Holding out: A8$/A4, B5@/B1, C4*/C2, D6@/D3
Epoch    0 end: loss = 208.998, acc = 0.580, test acc = 0.959
Epoch   50 end: loss = 110.484, acc = 0.967, test acc = 0.969
Epoch  100 end: loss = 109.628, acc = 0.969, test acc = 0.969
Epoch  150 end: loss = 109.076, acc = 0.969, test acc = 0.969
Epoch  200 end: loss = 108.980, acc = 0.969, test acc = 0.969
Epoch  250 end: loss = 108.994, acc = 0.969, test acc = 0.969
Epoch  300 end: loss = 108.914, acc = 0.969, test acc = 0.969
Epoch  350 end: loss = 108.637, acc = 0.969, test acc = 0.969
Epoch  400 end: loss =  94.725, acc = 0.932, test acc = 0.884
Epoch  450 end: loss =  83.528, acc = 0.882, test acc = 0.832
Epoch  500 end: loss =  72.968, acc = 0.875, test acc = 0.843
Epoch  550 end: loss =  64.154, acc = 0.875, test acc = 0.840
Epoch  600 end: loss =  57.884, acc = 0.882, test acc = 0.843
Epoch  650 end: loss =  53.357, acc = 0.899, test acc = 0.856
Epoch  700 end: loss =  49.506, ac

Troubleshooting, try with no testing

In [13]:
train_n_dd_nets(run_type='no_test', train_params={'do_combo_testing': False,
                                                  'holdout_testing': 'none'})

Training Iteration 1
---------------------
Epoch    0 end: loss = 205.913, acc = 0.598
Epoch  100 end: loss = 109.594, acc = 0.969
Epoch  200 end: loss = 108.824, acc = 0.969
Epoch  300 end: loss = 108.700, acc = 0.969
Epoch  400 end: loss = 108.665, acc = 0.969
Epoch  500 end: loss = 108.466, acc = 0.969
Epoch  600 end: loss =  97.810, acc = 0.950
Epoch  700 end: loss =  77.425, acc = 0.850
Epoch  800 end: loss =  68.138, acc = 0.868
Epoch  900 end: loss =  55.826, acc = 0.895
Epoch 1000 end: loss =  47.401, acc = 0.899
Epoch 1100 end: loss =  37.915, acc = 0.917
Epoch 1200 end: loss =  29.719, acc = 0.936
Epoch 1300 end: loss =  25.079, acc = 0.944
Epoch 1400 end: loss =  22.493, acc = 0.952
Epoch 1500 end: loss =  20.901, acc = 0.957
Epoch 1600 end: loss =  19.798, acc = 0.959
Epoch 1700 end: loss =  19.125, acc = 0.961
Epoch 1800 end: loss =  18.431, acc = 0.963
Epoch 1900 end: loss =  16.261, acc = 0.964
Epoch 2000 end: loss =  14.847, acc = 0.968
Epoch 2100 end: loss =  13.996, a

Base network with hold-out testing

In [ ]:
train_n_dd_nets(run_type='ho_both', train_params={'holdout_testing': 'all'})

Try holding out only item or context at a time

In [12]:
train_n_dd_nets(run_type='ho_item', train_params={'holdout_testing': 'item'})

Training Iteration 1
---------------------
Holding out item: A5@
Epoch    0 end: loss = 209.626, acc = 0.579, epochs for new item =   1279
Epoch  100 end: loss = 110.009, acc = 0.969
Epoch  200 end: loss = 109.327, acc = 0.969
Epoch  300 end: loss = 109.071, acc = 0.969
Epoch  400 end: loss = 103.845, acc = 0.968, epochs for new item =   1853
Epoch  500 end: loss =  87.080, acc = 0.901
Epoch  600 end: loss =  70.469, acc = 0.853
Epoch  700 end: loss =  57.883, acc = 0.870
Epoch  800 end: loss =  46.932, acc = 0.894, epochs for new item =   1021
Epoch  900 end: loss =  37.408, acc = 0.913
Epoch 1000 end: loss =  26.754, acc = 0.940
Epoch 1100 end: loss =  22.195, acc = 0.955
Epoch 1200 end: loss =  20.284, acc = 0.958, epochs for new item =    789
Epoch 1300 end: loss =  19.142, acc = 0.963
Epoch 1400 end: loss =  17.945, acc = 0.964
Epoch 1500 end: loss =  14.895, acc = 0.967
Epoch 1600 end: loss =  11.673, acc = 0.972, epochs for new item =    396
Epoch 1700 end: loss =  10.154, acc =

In [13]:
train_n_dd_nets(run_type='ho_context', train_params={'holdout_testing': 'context'})

Training Iteration 1
---------------------
Holding out context: C4
Epoch    0 end: loss = 211.982, acc = 0.569, epochs for new context =   1013
Epoch  100 end: loss = 108.563, acc = 0.968
Epoch  200 end: loss = 107.968, acc = 0.969
Epoch  300 end: loss = 107.663, acc = 0.969
Epoch  400 end: loss = 107.477, acc = 0.969, epochs for new context =    690
Epoch  500 end: loss =  96.449, acc = 0.964
Epoch  600 end: loss =  75.668, acc = 0.864
Epoch  700 end: loss =  57.252, acc = 0.892
Epoch  800 end: loss =  43.294, acc = 0.904, epochs for new context =    383
Epoch  900 end: loss =  31.842, acc = 0.919
Epoch 1000 end: loss =  22.180, acc = 0.943
Epoch 1100 end: loss =  15.670, acc = 0.954
Epoch 1200 end: loss =  11.994, acc = 0.968, epochs for new context =    299
Epoch 1300 end: loss =   9.862, acc = 0.976
Epoch 1400 end: loss =   8.450, acc = 0.981
Epoch 1500 end: loss =   7.399, acc = 0.984
Epoch 1600 end: loss =   6.634, acc = 0.986, epochs for new context =    304
Epoch 1700 end: loss

Try simple item tree, to see if separation b/w item classes still happens when they're all equally "typical"

Result: actually creates a really different pattern - no similarity b/w subgroups of different domains that exceeds similarity within a domain.

In [4]:
train_n_dd_nets(run_type='simplified_no_holdout', net_params={'simple_item_tree': True},
                train_params={'holdout_testing': 'none'})

Training Iteration 1
---------------------
Epoch    0 end: loss = 206.241, acc = 0.590
Epoch  100 end: loss = 109.462, acc = 0.968
Epoch  200 end: loss = 108.880, acc = 0.969
Epoch  300 end: loss = 108.726, acc = 0.969
Epoch  400 end: loss = 108.587, acc = 0.969
Epoch  500 end: loss = 108.492, acc = 0.969
Epoch  600 end: loss = 108.499, acc = 0.969
Epoch  700 end: loss = 108.346, acc = 0.969
Epoch  800 end: loss = 108.407, acc = 0.969
Epoch  900 end: loss = 108.298, acc = 0.969
Epoch 1000 end: loss = 108.313, acc = 0.969
Epoch 1100 end: loss = 108.229, acc = 0.969
Epoch 1200 end: loss = 108.169, acc = 0.969
Epoch 1300 end: loss = 108.237, acc = 0.969
Epoch 1400 end: loss =  88.505, acc = 0.942
Epoch 1500 end: loss =  67.182, acc = 0.872
Epoch 1600 end: loss =  48.577, acc = 0.904
Epoch 1700 end: loss =  43.639, acc = 0.918
Epoch 1800 end: loss =  42.378, acc = 0.919
Epoch 1900 end: loss =  39.936, acc = 0.921
Epoch 2000 end: loss =  35.435, acc = 0.933
Epoch 2100 end: loss =  28.163, a